바닐라 아이스크림이 가장 기본적인 맛을 가진 아이스크림인 것처럼, 앞서 배운 RNN을 가장 단순한 형태의 RNN이라고 하여 바닐라 RNN(Vanilla RNN)이라고 한다. (케라스에서는 SimpleRNN) 바닐라 RNN 이후 바닐라 RNN의 한계를 극복하기 위한 다양한 RNN의 변형이 나왔다. 이번에 배우게 될 LSTM도 그 중 하나다. 앞으로의 설명에서 LSTM과 비교하여 RNN을 언급하는 것은 전부 바닐라 RNN을 뜻한다.

# 1. 바닐라 RNN의 한계
---

![그림 1](./images/section2/그림_1.png)

바닐라 RNN은 출력 결과가 이전의 계산 결과에 의존한다. 하지만 바닐라 RNN은 **비교적 짧은 시퀀스(sequence)에 대해서만 효과를 보이는 단점**이 있다. **바닐라 RNN의 시점(time step)이 길어질 수록 계속된 가중치와의 곱으로 인해 정보량이 작아져 앞의 정보가 뒤로 충분히 전달되지 못하는 현상이 발생**하기 때문이다. 위 그림은 첫번째 입력값인 $x_1$의 정보량을 짙은 남색으로 표현했을 때, 시점이 지날수록 $x_1$의 정보량이 손실되어가는 과정을 색이 점차 얕아지는 것으로 표현한 것이다. 뒤로 갈수록 $x_1$의 정보량은 손실되고, 시점이 충분히 긴 상황에서는 $x_1$의 전체 정보에 대한 영향력은 거의 의미가 없을 수도 있다.

어쩌면 가장 중요한 정보가 시점의 앞 쪽에 위치할 수도 있다. RNN으로 만든 언어 모델이 다음 단어를 예측하는 과정을 생각해보자. 예를 들어 

>"모스크바에 여행을 왔는데 건물도 예쁘고 먹을 것도 맛있었어. 그런데 글쎄 직장 상사한테 전화가 왔어. 어디냐고 묻더라구 그래서 나는 말했지. 저 여행왔는데요. 여기 ___ 에요" 

빈칸의 단어를 예측하기 위해서는 장소 정보가 필요하다. 그런데 장소 정보에 해당되는 단어인 '모스크바'는 앞에 위치하고 있다. 바닐라 RNN에선 처음 시점에 나왔던 "모스크바"라는 정보가 시간이 흐를수록 정보량이 줄어들게 되어 빈칸을 예측하는 시점에 와선 영향력이 거의 없을 수 있다. 따라서 충분한 기억력을 가지고 있지 못한다면 다음 단어를 엉뚱하게 예측하게 될 것이다.

이를 **장기 의존성 문제(the problem of Long-Term Dependencies)**라고 한다.

<br/><br/>
# 2. 바닐라 RNN 내부 열어보기
---


![그림 2](./images/section2/그림_2.png)

위 그림은 바닐라 RNN의 내부 구조이다. 이 책에서는 RNN 계열의 인공 신경망의 그림에서는 편향 $b$를 생략한다. 위 그림에 편향 $b$를 그린다면  옆에 tanh로 향하는 또 하나의 입력선을 그리면 된다. 

시점 t에서 은닉 상태 $h_t$는 다음과 같은 식으로 구할 수 있다.

$$
h_t = \text{tanh}(W_hh_{t-1}+W_xx_t+b)
$$

<br/><br/>
# 3. LSTM(Long Short-Term Memory)
---

바닐라 RNN의 장기 의존성 문제를 보완한 RNN의 일종을 장단기 메모리(Long Short-Term Memory)라고 하며, 줄여서 LSTM이라고 한다. 

![그림 3](./images/section2/그림_3.png)

위 그림은 LSTM의 전체적인 내부의 모습이다. LSTM은 **은닉층의 메모리 셀에 입력 게이트, 망각 게이트, 출력 게이트를 추가하여 불필요한 기억을 지우고, 기억해야할 것들을 정한다.** LSTM은 은닉 상태(hidden state)를 계산하는 식이 바닐라 RNN보다 조금 더 복잡해졌고 **셀 상태(cell state)라는 값을 추가**했다. 위 그림에서는 t시점의 셀 상태를 $C_t$로 표현하고 있다. **LSTM은 RNN과 비교하여 긴 시퀀스의 입력을 처리하는데 탁월한 성능을 보인다.**

![그림 4](./images/section2/그림_4.png)

셀 상태는 위 그림에서 왼쪽에서 오른쪽으로 가는 굵은 선이다. 셀 상태 또한 이전에 배운 **은닉 상태처럼 이전 시점의 셀 상태가 다음 시점의 셀 상태를 구하기 위한 입력으로서 사용**됩니다.

은닉 상태의 값과 셀 상태의 값을 구하기 위해서 새로 추가 된 3개의 게이트를 사용한다. 각 게이트는 망각 게이트, 입력 게이트, 출력 게이트라고 부르며 이 3개의 게이트에는 공통적으로 시그모이드 함수가 존재한다. 시그모이드 함수를 지나면 0과 1사이의 값이 나오게 되는데 이 값들을 가지고 게이트를 조절한다. 아래의 내용을 참고로 각 게이트에 대해서 알아보자.

* $W_{xi},W_{xg},W_{xf},W_{xo}$는 $x_t$와 함께 각 게이트에서 사용되는 4개의 가중치이다.
* $W_{hi},W_{hg},W_{hf},W_{ho}$는 $h_{t-1}$와 함께 각 게이트에서 사용되는 4개의 가중치이다.
* $b_i,b_g,b_f,b_o$는 각 게이트에서 사용되는 4개의 편향이다.

여기서 i,g는 입력 게이트 단계에서 사용되고, f는 망각 게이트 단계, o는 출력 게이트 단계에서 사용된다. 

## 1) 입력 게이트(input gate)

![그림 5](./images/section2/그림_5.png)

$$
i_t=\sigma(W_{xi}x_t+W_{hi}h_{t-1}+b_i) \\
g_t = \text{tanh}(W_{xg}x_t+W_{hg}h_{t-1}+b_g)
$$

입력 게이트는 **현재 정보를 기억하기 위한 게이트**이다. 입력 게이트에선 위의 2가지 값들을 계산한다.

* $i_t$

    현재 시점 t의 값 $x_t$와 입력 게이트로 이어지는 가중치 $W_{xi}$를 곱한 값과 이전 시점 t-1의 은닉 상태 $h_{t-1}$와 입력 게이트로 이어지는 가중치 $W_{hi}$를 곱한 값을 더하여 시그모이드 함수를 지난다. 

* $g_t$

    현재 시점 t의 값 $x_t$와 입력 게이트로 이어지는 가중치 $W_{xg}$를 곱한 값과 이전 시점 t-1의 은닉 상태 $h_{t-1}$가 입력 게이트로 이어지는 가중치 $W_{hg}$를 곱한 값을 더하여 하이퍼볼릭탄젠트 함수를 지난다. 

시그모이드 함수를 지나 0과 1사이의 값을 가지는 $i_t$와 하이퍼볼릭탄젠트 함수를 지나 -1과 1사이의 값을 가지는 $g_t$. LSTM은 두 개의 값을 가지고 시점 t에서 선택된 기억할 정보의 양을 정한다. 구체적으로 어떻게 결정하는지는 아래에서 배우게 될 셀 상태 파트를 참고하자.

## 2) 망각 게이트(forget gate)

![그림 5](./images/section2/그림_5.png)

$$
f_t = \sigma(W_{xf}x_t+W_{hf}h_{t-1}+b_f)
$$

망각 게이트는 기억을 삭제하기 위한 게이트이다. 현재 시점 t의 값 $x_t$와 이전 시점 t-1의 은닉 상태$h_{t-1}$가 시그모이드 함수를 지나게 된다다. 시그모이드 함수를 지나면 0과 1 사이의 값이 나오게 되는데, **이 값이 곧 삭제 과정을 거친 정보의 양**이다. **0에 가까울수록 정보가 많이 삭제된 것이고 1에 가까울수록 정보를 온전히 기억한 것**이다. 이를 가지고 셀 상태를 구하게 되는데, 구체적으로는 아래에서 배우게 될 셀 상태 수식을 참고하자. 망각 게이트는 왜 필요한지 아직 잘 모르겠다.

## 3) 셀 상태(cell state)

![그림 7](./images/section2/그림_7.png)

$$
C_t = f_t \circ C_{t-1} + i_t \circ g_t
$$

시점 t의 셀 상태 $C_t$를 구하는 방법을 알아보자. 현재 망각 게이트에서 일부 기억을 잃은 상태이다.

1. 입력 게이트에서 구한 $i_t,g_t$ 의 값에 대해서 원소별 곱(element-wise product)을 진행한다. 여기서는 식으로 $\circ$ 로 표현한다. 이것이 **이번에 선택된 기억할 값**이다.<br/>
2. 망각 게이트에서 나온 값 $f_t$를 t-1 시점의 셀 상태와 원소별 곱을 진행한다.<br/>
3. 입력 게이트에서 선택된 기억을 2.의 결과값과 더한다. 이 값을 현재 시점 t의 셀 상태 $C_t$ 라고 하며, 이 값은 다음 t+1 시점의 LSTM 셀로 넘겨진다. <br/>

망각 게이트와 입력 게이트의 영향력을 이해해보자. 망각게이트의 값에 따라 극단적으로 2개의 망각 상태를 표현할 수 있다. 

1. 망각 게이트가 완전히 닫고 입력 게이트는 연 상태 <br/>  만약 삭제 게이트의 출력값인 $f_t$가 0이 된다면, $f_t$와 이전 시점의 셀 상태의 값 $C_{t-1}$의 원소별 곱 $f_t \circ C_{t-1}$ 은 0이 됨으로써 **현재 시점의 셀 상태의 값을 결정하기 위한 영향력이 0이 된다. 따라서 입력 게이트의 결과만이 현재 시점의 셀 상태의 값 $C_t$를 결정할 수 있다.**

2. 입력 게이트를 완전히 닫고 망각 게이트만을 연 상태 </br> 반대로 입력 게이트의 값을 0이라고 한다면, $i_t\circ g_t$는 0이 된다. 따라서 현재 시점의 셀 상태의 값 $C_t$는 오직 이전 시점의 셀 상태의 값 $C_{t-1}$에만 의존하게 된다. 

결과적으로 **망각 게이트는 이전 시점의 입력을 얼마나 반영할지를 의미하고, 입력 게이트는 현재 시점의 입력을 얼마나 반영할지를 결정**한다.

음.. 상상해보자면.. 각각의 시점 k 에서의 출력 값 $y_k$들에 의해서 error가 연산되어 역전파가 진행될 것인데, 올바른 예측을 하기 위해서 자연스럽게 입력/망각 게이트들이 학습되어 장기적인 메모리를 가져온다는 개념인 것 같다.

## 4) 출력 게이트와 은닉 상태

![그림 8](./images/section2/그림_8.png)

$$
o_t = \sigma(W_{xo}x_t+W_{ho}h_{t-1}+b_o)\\
h_t=o_t\circ\text{tanh}(C_t)
$$

출력 게이트는 현재 시점 t의 값 $x_t$와 이전 시점 t-1의 은닉 상태 $h_{t-1}$이 더해져 시그모이드 함수를 지난 값 $o_t$를 출력한다. $o_t$는 현재 시점 t의 은닉 상태 $h_t$ 를 결정하는 일에 쓰이게 된다. 셀 상태의 값 $C_t$가 하이퍼볼릭탄젠트 함수를 지나 -1과 1사이의 값이 되고, 출력 게이트의 값과 원소별 곱으로 연산되면서, 값이 걸러지는 효과가 발생하여 은닉 상태가 된다. 은닉 상태 값 $h_t$는 다음 시점과 출력층으로 이동한다.
